In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn

In [2]:
def getfiles(start="bhist",items='120',design='v30',k='k10',schemes=['fixed_express'],mis=0):
    listoffiles=[]
    where='Results/'
    kmap=getkmap()
    for file in os.listdir(where):
        if file.startswith(start):
            num=[x for x in schemes if x in file]
            if items in file and design in file and len(num)>0 and (file.endswith('mis.csv')==mis):
                if kmap[file.split('-')[1]]:
                    if k in file:
                        #listoffiles.append(file)
                        if file.endswith(k+'.csv'):
                            listoffiles.append(file)
                else:
                    #listoffiles.append(file)
                    if file.endswith(design+'.csv'):
                        listoffiles.append(file)
                
    return where,listoffiles

In [3]:
def getkmap():
    kmap={'TS':0,'TSe4':0,'TSe1':0,'TSe2':0,'TSthres':1,'TSdouble':1, 'greedy':0,'greedythres':1,
              'fixed_express':0,'fixed_sparse':0,'uncert':1,'mismin':1,
         'TSe4apprx':0,'TSthresapprx':1,'misminapprox':1,'TSregthres':1,
         'newTSthres':1,'newTSregthres':1,'TSe3apprx':0,'TSe3':0,'greedy3':0}
    return kmap

In [4]:
def histtableshort(items='120',k=10,design='v30',schemes=['fixed_express'],name='name.pdf',start=4,end=24,iters=100,mis=0):
    where,listoffiles=getfiles(start='bhist',items=items,k='k'+str(k),design=design,schemes=schemes,mis=mis)
    print(listoffiles)
    if items=='300':
        itemfile='HB_300items.csv'
        nitems=300
    elif items=='40':
        itemfile='HB_40items.csv'
        nitems=40
    elif items=='86':
        itemfile='ZB_86items.csv'
        nitems=86
    else:
        itemfile='HB_120items.csv'
        nitems=120
    data1=pd.read_csv(itemfile)
    utilsall=data1.get_values()[:,2:]
    realmean=np.mean(utilsall,0)
    topk=np.argsort(realmean)[::-1][:k]
    itemcol=[]
    nitems=int(items)
    for i in range(1,nitems+1):
        itemcol+=['Item_'+str(i)]
    resps=50
    topresp=np.zeros((iters,resps))
    resp=[x for x in range(20,1020,20)]
    meanvar=np.zeros((1,len(listoffiles)+1))
    cols=['k']
    for ind,file in enumerate(listoffiles):
        data=pd.read_csv(where+'/'+file)
        label=file.split('-')[1]
        for j in range(iters):

            iterdata=data[data['Iter']==(j+1)]
            for i,cur in enumerate(np.argsort(iterdata[itemcol].get_values())[:,::-1][:,:k]):
                topresp[j,i]=(len(set.intersection(set(cur),set(topk))))/k
        cols+=[label]
        topkmean=np.mean(topresp,0)

        #topkvar=np.var(topresp,0)
        meanvar[0,ind+1]=topkmean[end]
        #meanvar[1,ind+1]=topkvar[end]
    table=pd.DataFrame(meanvar)
    table.columns=cols
    table['k']=[str(k)]#,'Variance']
    return table

In [6]:
schemes1=['TS-','greedy-','TSe4-']
schemes2=['mismin-','uncert-','greedythres-','newTSthres-','newTSregthres-']
schemes3=['fixed_express-']
schemes4=['fixed_sparse-','TSdouble-']

schemes5=['TSe4apprx-','TSe4-']
schemes6=['TSthres-','TSthresapprx-']
schemes7=['mismin-','misminapprox-']

In [6]:
table1=histtableshort(design='v20',k=3,schemes=schemes1+schemes2+schemes3)
table2=histtableshort(design='v20',k=10,schemes=schemes1+schemes2+schemes3)
table3=histtableshort(design='v20',k=20,schemes=schemes1+schemes2+schemes3)
table4=histtableshort(design='v20',k=40,schemes=schemes1+schemes2+schemes3)

['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k3.csv', 'bhist-mismin-120v20k3.csv', 'bhist-newTSregthres-120v20k3.csv', 'bhist-newTSthres-120v20k3.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k3.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k10.csv', 'bhist-mismin-120v20k10.csv', 'bhist-newTSregthres-120v20k10.csv', 'bhist-newTSthres-120v20k10.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k10.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k20.csv', 'bhist-mismin-120v20k20.csv', 'bhist-newTSregthres-120v20k20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k20.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k40.csv', 'bhist-mismin-120v20k40.csv', 'bhist-newTSregthres-120v20k40.csv', 'bhist-newTSt

In [7]:
table=table1.append(table2).append(table3).append(table4)

In [8]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.78000,0.95000,0.933333,0.9500,0.8300,0.97000,0.9400,0.956667,0.943333
0,10,0.87700,0.92500,0.928000,0.9390,0.8920,0.95200,0.9330,0.952000,0.945000
0,20,0.86200,0.82750,0.913500,0.9275,0.8890,0.92300,0.8635,0.868500,0.919000
0,40,0.88725,0.84475,0.925500,0.9370,0.9115,0.94075,0.8045,0.850250,0.938000


In [15]:
table.to_latex(float_format="%.3f")

'\\begin{tabular}{llrrrrrrrrrrrrrrrrrr}\n\\toprule\n{} &   k &  fixed\\_express &  fixed\\_express &  fixed\\_express &  fixed\\_express &  fixed\\_express &  greedy &  greedythres &  mismin &  newTSregthres &  newTSthres &    TS &    TS &  TSe4 &  TSe4 &  TSe4 &  TSregthres &  TSthres &  uncert \\\\\n\\midrule\n0 &   3 &          0.177 &          0.780 &          0.677 &          0.710 &          0.673 &   0.950 &        0.933 &   0.950 &          0.830 &       0.970 & 0.940 & 0.910 & 0.957 & 0.900 & 0.923 &       0.917 &    0.947 &   0.943 \\\\\n0 &  10 &          0.829 &          0.877 &          0.869 &          0.884 &          0.864 &   0.925 &        0.928 &   0.939 &          0.892 &       0.952 & 0.933 & 0.904 & 0.952 & 0.902 & 0.915 &       0.940 &    0.947 &   0.945 \\\\\n0 &  20 &          0.858 &          0.862 &          0.861 &          0.862 &          0.854 &   0.827 &        0.914 &   0.928 &          0.889 &       0.923 & 0.863 & 0.836 & 0.868 & 0.829 & 0.835 &      

In [16]:
resp=[x for x in range(20,1020,20)]

In [17]:
resp[12]

260

In [9]:
table1=histtableshort(design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=12)
table2=histtableshort(design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=12)
table3=histtableshort(design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=12)
table4=histtableshort(design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=12)

['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k3.csv', 'bhist-mismin-120v20k3.csv', 'bhist-newTSregthres-120v20k3.csv', 'bhist-newTSthres-120v20k3.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k3.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k10.csv', 'bhist-mismin-120v20k10.csv', 'bhist-newTSregthres-120v20k10.csv', 'bhist-newTSthres-120v20k10.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k10.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k20.csv', 'bhist-mismin-120v20k20.csv', 'bhist-newTSregthres-120v20k20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-uncert-120v20k20.csv']
['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-greedythres-120v20k40.csv', 'bhist-mismin-120v20k40.csv', 'bhist-newTSregthres-120v20k40.csv', 'bhist-newTSt

In [10]:
table=table1.append(table2).append(table3).append(table4)

In [11]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.65000,0.853333,0.860,0.883333,0.783333,0.863333,0.863333,0.873333,0.84000
0,10,0.82500,0.880000,0.902,0.907000,0.866000,0.919000,0.923000,0.909000,0.90100
0,20,0.82450,0.784000,0.863,0.884500,0.859500,0.882000,0.825500,0.820000,0.88550
0,40,0.85725,0.809000,0.884,0.908000,0.886750,0.900750,0.786750,0.818250,0.89675


In [22]:
table.to_latex(float_format="%.3f")

'\\begin{tabular}{llrrrrrrrrr}\n\\toprule\n{} &   k &  fixed\\_express &  greedy &  greedythres &  mismin &    TS &  TSe4 &  TSregthres &  TSthres &  uncert \\\\\n\\midrule\n0 &   3 &          0.650 &   0.853 &        0.860 &   0.883 & 0.863 & 0.873 &       0.850 &    0.893 &   0.840 \\\\\n0 &  10 &          0.825 &   0.880 &        0.902 &   0.907 & 0.923 & 0.909 &       0.909 &    0.907 &   0.901 \\\\\n0 &  20 &          0.824 &   0.784 &        0.863 &   0.885 & 0.825 & 0.820 &       0.869 &    0.879 &   0.886 \\\\\n0 &  40 &          0.857 &   0.809 &        0.884 &   0.908 & 0.787 & 0.818 &       0.895 &    0.888 &   0.897 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [12]:
table1=histtableshort(items='300',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=12)
table2=histtableshort(items='300',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=12)
table3=histtableshort(items='300',design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=12)
table4=histtableshort(items='300',design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=12)
table=table1.append(table2).append(table3).append(table4)

['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k3.csv', 'bhist-mismin-300v20k3.csv', 'bhist-newTSregthres-300v20k3.csv', 'bhist-newTSthres-300v20k3.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k3.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k10.csv', 'bhist-mismin-300v20k10.csv', 'bhist-newTSregthres-300v20k10.csv', 'bhist-newTSthres-300v20k10.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k10.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k20.csv', 'bhist-mismin-300v20k20.csv', 'bhist-newTSregthres-300v20k20.csv', 'bhist-newTSthres-300v20k20.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k20.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k40.csv', 'bhist-mismin-300v20k40.csv', 'bhist-newTSregthres-300v20k40.csv', 'bhist-newTSt

In [13]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.306667,0.593333,0.476667,0.543333,0.536667,0.646667,0.596667,0.640,0.580
0,10,0.462000,0.612000,0.540000,0.682000,0.576000,0.674000,0.630000,0.662,0.694
0,20,0.550500,0.599500,0.593500,0.722000,0.590000,0.714000,0.606500,0.658,0.719
0,40,0.617000,0.571750,0.616500,0.705000,0.638000,0.702250,0.525250,0.625,0.721


In [23]:
table.to_latex(float_format="%.3f")

'\\begin{tabular}{llrrrrrrrrr}\n\\toprule\n{} &   k &  fixed\\_express &  greedy &  greedythres &  mismin &  newTSregthres &  newTSthres &    TS &  TSe4 &  uncert \\\\\n\\midrule\n0 &   3 &          0.387 &   0.763 &        0.747 &   0.773 &          0.617 &       0.800 & 0.720 & 0.793 &   0.777 \\\\\n0 &  10 &          0.578 &   0.759 &        0.718 &   0.794 &          0.626 &       0.792 & 0.702 & 0.784 &   0.792 \\\\\n0 &  20 &          0.676 &   0.718 &        0.764 &   0.842 &          0.633 &       0.821 & 0.673 & 0.764 &   0.846 \\\\\n0 &  40 &          0.717 &   0.649 &        0.751 &   0.813 &          0.693 &       0.799 & 0.575 & 0.696 &   0.828 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [6]:
table1=histtableshort(items='300',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=24)
table2=histtableshort(items='300',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=24)
table3=histtableshort(items='300',design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=24)
table4=histtableshort(items='300',design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=24)
table=table1.append(table2).append(table3).append(table4)

['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k3.csv', 'bhist-mismin-300v20k3.csv', 'bhist-newTSregthres-300v20k3.csv', 'bhist-newTSthres-300v20k3.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k3.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k10.csv', 'bhist-mismin-300v20k10.csv', 'bhist-newTSregthres-300v20k10.csv', 'bhist-newTSthres-300v20k10.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k10.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k20.csv', 'bhist-mismin-300v20k20.csv', 'bhist-newTSregthres-300v20k20.csv', 'bhist-newTSthres-300v20k20.csv', 'bhist-TS-300v20.csv', 'bhist-TSe4-300v20.csv', 'bhist-uncert-300v20k20.csv']
['bhist-fixed_express-300v20.csv', 'bhist-greedy-300v20.csv', 'bhist-greedythres-300v20k40.csv', 'bhist-mismin-300v20k40.csv', 'bhist-newTSregthres-300v20k40.csv', 'bhist-newTSt

In [7]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.386667,0.763333,0.746667,0.773333,0.616667,0.8000,0.7200,0.793333,0.776667
0,10,0.578000,0.759000,0.718000,0.794000,0.626000,0.7920,0.7020,0.784000,0.792000
0,20,0.676000,0.718000,0.764500,0.841500,0.633000,0.8210,0.6735,0.764000,0.846500
0,40,0.716750,0.648750,0.751000,0.812500,0.692750,0.7995,0.5745,0.696250,0.827500


In [26]:
table.to_latex(float_format="%.3f")

'\\begin{tabular}{llrrrrrrrrr}\n\\toprule\n{} &   k &  fixed\\_express &  greedy &  greedythres &  mismin &  newTSregthres &  newTSthres &    TS &  TSe4 &  uncert \\\\\n\\midrule\n0 &   3 &          0.307 &   0.593 &        0.477 &   0.543 &          0.537 &       0.647 & 0.597 & 0.640 &   0.580 \\\\\n0 &  10 &          0.462 &   0.612 &        0.540 &   0.682 &          0.576 &       0.674 & 0.630 & 0.662 &   0.694 \\\\\n0 &  20 &          0.550 &   0.599 &        0.593 &   0.722 &          0.590 &       0.714 & 0.606 & 0.658 &   0.719 \\\\\n0 &  40 &          0.617 &   0.572 &        0.616 &   0.705 &          0.638 &       0.702 & 0.525 & 0.625 &   0.721 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [8]:
table1=histtableshort(items='40',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=12)
table2=histtableshort(items='40',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=12)
table=table1.append(table2)

['bhist-fixed_express-40v20.csv', 'bhist-greedy-40v20.csv', 'bhist-greedythres-40v20k3.csv', 'bhist-mismin-40v20k3.csv', 'bhist-newTSregthres-40v20k3.csv', 'bhist-newTSthres-40v20k3.csv', 'bhist-TS-40v20.csv', 'bhist-TSe4-40v20.csv', 'bhist-uncert-40v20k3.csv']
['bhist-fixed_express-40v20.csv', 'bhist-greedy-40v20.csv', 'bhist-greedythres-40v20k10.csv', 'bhist-mismin-40v20k10.csv', 'bhist-newTSregthres-40v20k10.csv', 'bhist-newTSthres-40v20k10.csv', 'bhist-TS-40v20.csv', 'bhist-TSe4-40v20.csv', 'bhist-uncert-40v20k10.csv']


In [9]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.896667,0.973333,0.970,0.940,0.970,0.956667,0.973333,0.973333,0.943333
0,10,0.869000,0.897000,0.912,0.907,0.913,0.906000,0.926000,0.905000,0.916000


In [10]:
table.to_latex(float_format="%.3f")

'\\begin{tabular}{llrrrrrrrrr}\n\\toprule\n{} &   k &  fixed\\_express &  greedy &  greedythres &  mismin &  newTSregthres &  newTSthres &    TS &  TSe4 &  uncert \\\\\n\\midrule\n0 &   3 &          0.897 &   0.973 &        0.970 &   0.940 &          0.970 &       0.957 & 0.973 & 0.973 &   0.943 \\\\\n0 &  10 &          0.869 &   0.897 &        0.912 &   0.907 &          0.913 &       0.906 & 0.926 & 0.905 &   0.916 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [13]:
table1=histtableshort(items='40',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=24)
table2=histtableshort(items='40',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=24)

table=table1.append(table2)

['bhist-fixed_express-40v20.csv', 'bhist-greedy-40v20.csv', 'bhist-greedythres-40v20k3.csv', 'bhist-mismin-40v20k3.csv', 'bhist-newTSregthres-40v20k3.csv', 'bhist-newTSthres-40v20k3.csv', 'bhist-TS-40v20.csv', 'bhist-TSe4-40v20.csv', 'bhist-uncert-40v20k3.csv']
['bhist-fixed_express-40v20.csv', 'bhist-greedy-40v20.csv', 'bhist-greedythres-40v20k10.csv', 'bhist-mismin-40v20k10.csv', 'bhist-newTSregthres-40v20k10.csv', 'bhist-newTSthres-40v20k10.csv', 'bhist-TS-40v20.csv', 'bhist-TSe4-40v20.csv', 'bhist-uncert-40v20k10.csv']


In [14]:
table

,k,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.930,0.990,0.990,0.983333,0.983333,0.976667,0.990,0.986667,0.990
0,10,0.903,0.919,0.933,0.925000,0.925000,0.920000,0.936,0.936000,0.932


In [24]:
table1=histtableshort(design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=12,mis=1)


['bhist-fixed_express-120v20k3-mis.csv', 'bhist-fixed_express-120v20test-mis.csv', 'bhist-fixed_express-120v20test1-mis.csv', 'bhist-greedy-120v20-mis.csv', 'bhist-greedythres-120v20k3-mis.csv', 'bhist-mismin-120v20k3-mis.csv', 'bhist-newTSregthres-120v20k3-mis.csv', 'bhist-newTSthres-120v20k3-mis.csv', 'bhist-TS-120v20-mis.csv', 'bhist-TSe4-120v20-mis.csv', 'bhist-uncert-120v20k3-mis.csv']


In [25]:
table1

,k,fixed_express,fixed_express,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.08,0.09,0.066667,0.296667,0.236667,0.253333,0.066667,0.356667,0.066667,0.346667,0.27


In [26]:
table2=histtableshort(design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=24,mis=1)

['bhist-fixed_express-120v20k3-mis.csv', 'bhist-fixed_express-120v20test-mis.csv', 'bhist-fixed_express-120v20test1-mis.csv', 'bhist-greedy-120v20-mis.csv', 'bhist-greedythres-120v20k3-mis.csv', 'bhist-mismin-120v20k3-mis.csv', 'bhist-newTSregthres-120v20k3-mis.csv', 'bhist-newTSthres-120v20k3-mis.csv', 'bhist-TS-120v20-mis.csv', 'bhist-TSe4-120v20-mis.csv', 'bhist-uncert-120v20k3-mis.csv']


In [27]:
table2

,k,fixed_express,fixed_express,fixed_express,greedy,greedythres,mismin,newTSregthres,newTSthres,TS,TSe4,uncert
0,3,0.23,0.196667,0.246667,0.57,0.49,0.54,0.1,0.64,0.126667,0.536667,0.503333


In [10]:
table1=histtableshort(items='86',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=12)
table2=histtableshort(items='86',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=12)
table3=histtableshort(items='86',design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=12)
table4=histtableshort(items='86',design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=12)
table=table1.append(table2).append(table3).append(table4)

['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-mismin-86v20k3.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k3.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k10.csv', 'bhist-mismin-86v20k10.csv', 'bhist-newTSregthres-86v20k10.csv', 'bhist-newTSthres-86v20k10.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k10.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k20.csv', 'bhist-mismin-86v20k20.csv', 'bhist-newTSregthres-86v20k20.csv', 'bhist-newTSthres-86v20k20.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k20.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k40.csv', 'bhist-newTSthres-86v20k40.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv']


In [11]:
table

,TS,TSe4,fixed_express,greedy,greedythres,k,mismin,newTSregthres,newTSthres,uncert
0,0.916667,0.95000,0.796667,0.9400,NaN,3,0.923333,NaN,NaN,0.933333
0,0.894000,0.89300,0.834000,0.8800,0.8920,10,0.899000,0.8730,0.89700,0.899000
0,0.839500,0.86950,0.868500,0.8245,0.9205,20,0.932000,0.8975,0.91800,0.916000
0,0.779500,0.86125,0.896500,0.8520,0.9175,40,NaN,NaN,0.92225,NaN


In [7]:
table1=histtableshort(items='86',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=24)
table2=histtableshort(items='86',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=24)
table3=histtableshort(items='86',design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=24)
table4=histtableshort(items='86',design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=24)
table=table1.append(table2).append(table3).append(table4)

['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-mismin-86v20k3.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k3.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k10.csv', 'bhist-mismin-86v20k10.csv', 'bhist-newTSregthres-86v20k10.csv', 'bhist-newTSthres-86v20k10.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k10.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k20.csv', 'bhist-mismin-86v20k20.csv', 'bhist-newTSregthres-86v20k20.csv', 'bhist-newTSthres-86v20k20.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k20.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k40.csv', 'bhist-mismin-86v20k40.csv', 'bhist-newTSregthres-86v20k40.csv', 'bhist-newTSthres-86v20k40.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k40.csv']


In [8]:
table

,TS,TSe4,fixed_express,greedy,greedythres,k,mismin,newTSregthres,newTSthres,uncert
0,0.97000,0.99000,0.903333,0.996667,NaN,3,0.983333,NaN,NaN,0.97000
0,0.90700,0.91800,0.876000,0.904000,0.9190,10,0.925000,0.89000,0.9120,0.91700
0,0.86400,0.91450,0.907000,0.873500,0.9580,20,0.961000,0.92150,0.9580,0.95100
0,0.79125,0.88825,0.924000,0.882750,0.9485,40,0.955250,0.94375,0.9535,0.95425


In [9]:
table1=histtableshort(items='86',design='v20',k=3,schemes=schemes1+schemes2+schemes3,end=49)
table2=histtableshort(items='86',design='v20',k=10,schemes=schemes1+schemes2+schemes3,end=49)
table3=histtableshort(items='86',design='v20',k=20,schemes=schemes1+schemes2+schemes3,end=49)
table4=histtableshort(items='86',design='v20',k=40,schemes=schemes1+schemes2+schemes3,end=49)
table=table1.append(table2).append(table3).append(table4)

['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-mismin-86v20k3.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k3.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k10.csv', 'bhist-mismin-86v20k10.csv', 'bhist-newTSregthres-86v20k10.csv', 'bhist-newTSthres-86v20k10.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k10.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k20.csv', 'bhist-mismin-86v20k20.csv', 'bhist-newTSregthres-86v20k20.csv', 'bhist-newTSthres-86v20k20.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k20.csv']
['bhist-fixed_express-86v20.csv', 'bhist-greedy-86v20.csv', 'bhist-greedythres-86v20k40.csv', 'bhist-mismin-86v20k40.csv', 'bhist-newTSregthres-86v20k40.csv', 'bhist-newTSthres-86v20k40.csv', 'bhist-TS-86v20.csv', 'bhist-TSe4-86v20.csv', 'bhist-uncert-86v20k40.csv']


In [10]:
table

,TS,TSe4,fixed_express,greedy,greedythres,k,mismin,newTSregthres,newTSthres,uncert
0,0.9800,1.000,0.943333,0.996667,NaN,3,0.996667,NaN,NaN,0.99000
0,0.9170,0.932,0.910000,0.923000,0.93400,10,0.927000,0.9040,0.931,0.93500
0,0.8870,0.942,0.943000,0.904000,0.98300,20,0.981000,0.9440,0.980,0.97950
0,0.8075,0.915,0.945250,0.917500,0.96925,40,0.973000,0.9605,0.972,0.97225
